In [2]:
import numpy as np
import simpy
from scipy.stats import expon, gamma, uniform

In [ ]:
# priority resources: cut off the lower priority people in the queue, but do not kick off people in the service.
# preempt resources: when you at front line, can kick off the people in service with higher priority but not preemptive

In [7]:
def simulate(lam_dt, lam_st, T):

    def arrivals():
        while True:
            dt = expon.rvs(scale=1/lam_dt)
            if env.now + dt < T:
                yield env.timeout(dt)
                env.process(service())
            else:
                break
    
    def service():
        t0 = env.now
        if uniform.rvs() < 0.2:
            priority = 0
            preempt = True
        else:
            priority = 1
            preempt = False
        rqt = server.request(priority=priority, preempt=preempt)
        yield rqt
        st = expon.rvs(scale=1/lam_st)
        yield env.timeout(st)
        server.release(rqt)
        X.append(env.now - t0)

    X = []
    env = simpy.Environment()
    server = simpy.PreemptiveResource(capacity=1, env=env)
    env.process(arrivals())
    env.run()
    return np.max(X)


In [8]:
simulate(10, 1, 10) 
# In SimPy, an 'Interrupt' occurs when a higher-priority process preempts a lower-priority process. 

Interrupt: Interrupt(<simpy.resources.resource.Preempted object at 0x000001F78ACAEEF0>)

In [10]:
def simulate(lam_dt, lam_st, T):

    def arrivals():
        while True:
            dt = expon.rvs(scale=1/lam_dt)
            if env.now + dt < T:
                yield env.timeout(dt)
                env.process(service())
            else:
                break
    
    def service():
        t0 = env.now
        if uniform.rvs() < 0.2:
            priority = 0
            preempt = True
        else:
            priority = 2
            preempt = False
        rqt = server.request(priority=priority, preempt=preempt)
        yield rqt
        st = expon.rvs(scale=1/lam_st)
        while True:
            try:
                yield env.timeout(st)
                server.release(rqt)
                break
            except:
                rqt = server.request(priority=1, preempt=False)
                yield rqt
# If the service time (st) is completed, it releases the server. 
# If an exception is raised (likely indicating preemption), 
# it re-requests the server with lower priority and non-preemptive behavior until the service is completed.

# If the preempted customer is preempted again before completing its service 
# (i.e., an exception is raised within the try block), 
# it re-requests the server with a lower priority (priority=1) and non-preemptive behavior (preempt=False). 
# This means that the preempted customer will not preempt other customers in the queue, 
# and it will wait for its turn to receive service.
        X.append(env.now - t0)

    X = []
    env = simpy.Environment()
    server = simpy.PreemptiveResource(capacity=1, env=env)
    env.process(arrivals())
    env.run()
    return np.max(X)


In [11]:
simulate(10, 1, 10) 

83.1866474311745

In [12]:
def s1():
    yield env.timeout(0.5)
    yield env.timeout(0.5)
    
def s2(s):
    yield env.timeout(0.6)
    s.interrupt()

env = simpy.Environment()
s = env.process(s1())
env.process(s2(s))
env.run()


Interrupt: Interrupt(None)

In [15]:
def s1():
    yield env.timeout(0.5)
    try:
        yield env.timeout(0.5)
    except:
        print("Oh no! I was interrupted by the other process")
        print(env.now)
        yield env.timeout(0.5)
        print(env.now)

def s2(s):
    try:
        yield env.timeout(1.5)
        s.interrupt()
    except:
        print('I couldnt interrupt process 1')

env = simpy.Environment()
s = env.process(s1())
env.process(s2(s))
env.run()


I couldnt interrupt process 1


In [17]:
### let's say we have a single server queue
### homogeneous poisson process arrivals
### service time is exponential
### after being in line for 0.5, they get a phone call, and will leave and not come back if they
### are in line, but will not leave if they are in service
### rqt.processed=True

lam_dt = 5
lam_st = 5
T = 10

def arrivals():
    while True:
        dt = expon.rvs(scale=1/lam_dt)
        if env.now + dt < T:
            yield env.timeout(dt)
            env.process(service())
        else:
            break


def service():
    rqt = server.request()
    yield rqt
    st = expon.rvs(scale=1/lam_st)
    yield env.timeout(st)
    server.release(rqt)



env = simpy.Environment()
server = simpy.Resource(env=env, capacity=1)
env.process(arrivals())
env.run()



In [16]:
### let's say we have a single server queue
### homogeneous poisson process arrivals
### service time is exponential
### after being in line for 0.5, they get a phone call, and will leave and not come back if they
### are in line, but will not leave if they are in service
### rqt.processed=True

lam_dt = 5
lam_st = 5
T = 10

def arrivals():
    while True:
        dt = expon.rvs(scale=1/lam_dt)
        if env.now + dt < T:
            yield env.timeout(dt)
            rqt = server.request()
            svc = env.process(service(rqt))
            env.process(call(svc, rqt)) # in parallel, have a call process
        else:
            break

def call(svc, rqt):
    yield env.timeout(0.5)
    if rqt.processed == False: # whether the rqt gone to the service
        svc.interrupt()
        print('someone got a call and left')

def service(rqt):
    #rqt = server.request()
    try: # someone may be got interrupted while waiting for the service
        yield rqt
        st = expon.rvs(scale=1/lam_st)
        yield env.timeout(st)
        server.release(rqt)
    except:
        rqt.cancel()



env = simpy.Environment()
server = simpy.Resource(env=env, capacity=1)
env.process(arrivals())
env.run()



someone got a call and left
someone got a call and left
someone got a call and left
someone got a call and left
someone got a call and left
someone got a call and left
someone got a call and left
someone got a call and left
someone got a call and left
someone got a call and left
someone got a call and left
someone got a call and left
someone got a call and left


In [18]:
### change "line" to "system"
### let's say we have a single server queue
### homogeneous poisson process arrivals
### service time is exponential
### after being in system for 0.5, they get a phone call, and will leave and not come back
### rqt.processed=True

lam_dt = 5
lam_st = 1
T = 10

def arrivals():
    while True:
        dt = expon.rvs(scale=1/lam_dt)
        if env.now + dt < T:
            yield env.timeout(dt)
            svc = env.process(service())
            env.process(call(svc,))
        else:
            break

def call(svc):
    yield env.timeout(0.5) # do not need to check the rqt.processed condition
    try: 
        svc.interrupt()
    except:
        pass

def service():
    rqt = server.request()
    try:
        yield rqt
        st = expon.rvs(scale=1/lam_st)
        yield env.timeout(st)
        server.release(rqt)
        print('someone completed service')
    except:
        if rqt.processed == False:
            rqt.cancel()
            print('someone got a call and left while in line')
        else:
            server.release(rqt)
            print('someone got a call and left while in service')



env = simpy.Environment()
server = simpy.Resource(env=env, capacity=1)
env.process(arrivals())
env.run()



someone completed service
someone completed service
someone got a call and left while in service
someone got a call and left while in service
someone got a call and left while in service
someone got a call and left while in service
someone got a call and left while in service
someone got a call and left while in service
someone completed service
someone completed service
someone completed service
someone completed service
someone completed service
someone got a call and left while in service
someone got a call and left while in service
someone completed service
someone got a call and left while in service
someone got a call and left while in service
someone got a call and left while in service
someone got a call and left while in service
someone got a call and left while in service
someone got a call and left while in service
someone got a call and left while in service
someone got a call and left while in service
someone got a call and left while in service
someone got a call and left

In [ ]:
### let's say we have a single server queue
### homogeneous poisson process arrivals
### service time is exponential
### after being in system for an exponential amount of time, they get a phone call, 
### and will leave and not come back
### rqt.processed=True

lam_dt = 5
lam_st = 1
T = 10

def arrivals():
    i = 0
    while True:
        i += 1
        dt = expon.rvs(scale=1/lam_dt)
        if env.now + dt < T:
            yield env.timeout(dt)
            svc = env.process(service(i))
            env.process(call(svc,i))
        else:
            break

def call(svc,i):
    yield env.timeout(expon.rvs(scale=1))    
    try:
        svc.interrupt()
    except:
        pass

def service(i):
    rqt = server.request()
    try:
        yield rqt
        st = expon.rvs(scale=1/lam_st)
        yield env.timeout(st)
        server.release(rqt)
        print('person {} completed service at {}'.format(i, env.now))
    except:
        if rqt.processed == False:
            rqt.cancel()
            print('{} got a call and left while in line at {}'.format(i, env.now))
        else:
            server.release(rqt)
            print('{} got a call and left while in service at {}'.format(i, env.now))



env = simpy.Environment()
server = simpy.Resource(env=env, capacity=1)
env.process(arrivals())
env.run()